In [1]:
import pandas as pd
df = pd.read_csv('./data/CRVUSDT_15m.csv')
df.head()

,time,open,high,low,close,volume
0,2023-08-01 00:00:00,0.566,0.567,0.556,0.559,14858286.8
1,2023-08-01 00:15:00,0.558,0.568,0.558,0.562,12217157.4
2,2023-08-01 00:30:00,0.563,0.571,0.560,0.566,11195388.2
3,2023-08-01 00:45:00,0.567,0.568,0.560,0.562,8884700.7
4,2023-08-01 01:00:00,0.563,0.563,0.558,0.562,8656179.8


In [2]:
import pandas_ta as ta
bb = ta.bbands(df.close, length=30, std=1.5)
#add all to df
df = df.join(bb)
df['ATR']= df.ta.atr()
#help(ta.adx)

In [3]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,BBL_30_1.5,BBM_30_1.5,BBU_30_1.5,BBB_30_1.5,BBP_30_1.5,ATR
29,2023-08-01 07:15:00,0.576,0.584,0.566,0.581,38798559.7,0.495679,0.538233,0.580788,15.812733,1.002491,0.021723
30,2023-08-01 07:30:00,0.581,0.589,0.576,0.585,35235749.1,0.495044,0.539100,0.583156,16.344157,1.020932,0.021024
31,2023-08-01 07:45:00,0.585,0.591,0.575,0.577,36859647.5,0.494781,0.539600,0.584419,16.611923,0.917234,0.020625
32,2023-08-01 08:00:00,0.577,0.582,0.570,0.580,42795919.1,0.494477,0.540067,0.585656,16.882865,0.937968,0.019946
33,2023-08-01 08:15:00,0.579,0.618,0.579,0.593,104531954.2,0.493665,0.541100,0.588535,17.532699,1.047068,0.021436


In [4]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

C:\Users\hiban\AppData\Local\Temp\ipykernel_28676\1852176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


,open,high,low,close,volume,BBL_30_1.5,BBM_30_1.5,BBU_30_1.5,BBB_30_1.5,BBP_30_1.5,ATR
time,,,,,,,,,,,
2023-08-01 07:15:00,0.576,0.584,0.566,0.581,38798559.7,0.495679,0.538233,0.580788,15.812733,1.002491,0.021723
2023-08-01 07:30:00,0.581,0.589,0.576,0.585,35235749.1,0.495044,0.539100,0.583156,16.344157,1.020932,0.021024
2023-08-01 07:45:00,0.585,0.591,0.575,0.577,36859647.5,0.494781,0.539600,0.584419,16.611923,0.917234,0.020625
2023-08-01 08:00:00,0.577,0.582,0.570,0.580,42795919.1,0.494477,0.540067,0.585656,16.882865,0.937968,0.019946
2023-08-01 08:15:00,0.579,0.618,0.579,0.593,104531954.2,0.493665,0.541100,0.588535,17.532699,1.047068,0.021436


In [5]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [6]:
def bollinger_doji_signal(df): 
    #bullish signal
    if ( df.Close.iloc[-1] < df['BBL_30_1.5'].iloc[-1] and
       #df.Close.iloc[-1] > df['BBL_30_1.5'].iloc[-1] and df.Close.iloc[-1] < df['BBM_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] > df.Open.iloc[-1] and
       df.Close.iloc[-2] == df.Open.iloc[-2] and
       df.Close.iloc[-3] < df.Open.iloc[-3] ):
        return 2
    
    #bearish signal
    elif ( df.Close.iloc[-1] > df['BBU_30_1.5'].iloc[-1] and
        #df.Close.iloc[-1] < df['BBU_30_1.5'].iloc[-1] and df.Close.iloc[-1] > df['BBM_30_1.5'].iloc[-1] and
       df.Close.iloc[-1] < df.Open.iloc[-1] and
       df.Close.iloc[-2] == df.Open.iloc[-2] and
       df.Close.iloc[-3] > df.Open.iloc[-3] ):
         return 1
    
    #nosignal
    else:
        return 0

signal = [0]*len(df)
for i in range(20,len(df)):
    df_temp = df[i-3:i+1]
    signal[i]= bollinger_doji_signal(df_temp)
df["bollinger_doji_signal"] = signal

In [7]:
import numpy as np
def pointpos(x):
    if x['bollinger_doji_signal']==1:
        return x['High']+0.5e-3
    elif x['bollinger_doji_signal']==2:
        return x['Low']-0.5e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [8]:
df['bollinger_doji_signal'].value_counts()

0    7847
1      18
2      12
Name: bollinger_doji_signal, dtype: int64

In [9]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class doji_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'bollinger_doji_signal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        if self.signal == 2 :
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            self.buy(sl=sl_price, tp=tp_price)
        if self.signal == 1 :
            sl_price = self.data.Close[-1] + slatr
            tp_price = self.data.Close[-1] - self.TPSL_rate * slatr
            self.sell(sl=sl_price, tp=tp_price)

bt = Backtest(df, doji_Strategy, cash=100, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2023-08-01 07:15:00
End                       2023-10-22 08:15:00
Duration                     82 days 01:00:00
Exposure Time [%]                    4.430621
Equity Final [$]                    65.852893
Equity Peak [$]                    166.548332
Return [%]                         -34.147107
Buy & Hold Return [%]              -24.956971
Return (Ann.) [%]                  -84.071806
Volatility (Ann.) [%]              315.293674
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -81.002785
Avg. Drawdown [%]                  -19.721244
Max. Drawdown Duration       38 days 14:30:00
Avg. Drawdown Duration        5 days 13:04:00
# Trades                                   30
Win Rate [%]                        46.666667
Best Trade [%]                       2.052662
Worst Trade [%]                     -1.697158
Avg. Trade [%]                    

In [10]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [11]:
%%time
stats = bt.optimize(atr_sl_rate = np.arange(0.5, 2.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.5, 2.6, 0.2).tolist(),
                    maximize='Equity Final [$]')
stats

  0%|          | 0/13 [00:00<?, ?it/s]

CPU times: total: 11.7 s
Wall time: 20.1 s


Start                     2023-08-01 07:15:00
End                       2023-10-22 08:15:00
Duration                     82 days 01:00:00
Exposure Time [%]                    4.164022
Equity Final [$]                    225.88854
Equity Peak [$]                     225.88854
Return [%]                          125.88854
Buy & Hold Return [%]              -24.956971
Return (Ann.) [%]                   3499.8277
Volatility (Ann.) [%]            99327.945969
Sharpe Ratio                         0.035235
Sortino Ratio                       22.355074
Calmar Ratio                        43.285522
Max. Drawdown [%]                  -80.854464
Avg. Drawdown [%]                  -18.142405
Max. Drawdown Duration       32 days 07:45:00
Avg. Drawdown Duration        4 days 13:24:00
# Trades                                   30
Win Rate [%]                             70.0
Best Trade [%]                       1.732277
Worst Trade [%]                     -1.915499
Avg. Trade [%]                    

In [12]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [13]:
stats._strategy

<Strategy doji_Strategy(atr_sl_rate=1.8999999999999997,TPSL_rate=0.7)>

In [14]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'])])
fig.show()

In [15]:
#heatmap plotly

import plotly.express as px
import plotly.graph_objects as go

#trades
fig = go.Figure(data=go.Heatmap(
                     z=stats['_trades']['PnL'],
                     x=stats['_trades']['EntryTime'],
                     y=stats['_trades']['EntryPrice'],
                     hoverongaps = False))
fig.show()